# Problem Statement

Functionalities to backup REDCap data in the database is constantly being maintained and updated. There needs to be a way to quickly verify that the data in the backup database is the same as the data on REDCap.

This comparison goes both ways--data found in REDCap should be in the database, and data found in the database should be on REDCap.

# Solution

This notebook pulls data from REDCap and the database and do cross checking. Then, differences (if any) are exported to an excel file.

# Code
### 1. Setup: get project's records and database's data

In [91]:
import AMBRA_Backups
import pandas as pd
import logging
from datetime import datetime
from redcap import Project
from AMBRA_Backups.Database.database import Database
from AMBRA_Backups.redcap_funcs import get_project_instru_field_map
logger = logging.getLogger(__name__)

In [2]:
project_name = 'TESTED DC'
db_name = 'TESTED'

Get project's data

In [ ]:
project = AMBRA_Backups.redcap_funcs.get_redcap_project(project_name)

In [ ]:
def get_project_records(project: Project):
    '''Get records from REDCap project'''
    project_records = project.export_records(format_type='df')
    project_records.reset_index(inplace=True)
    project_records.to_excel(f'{project_name}_redcap_original.xlsx')
    return project_records

In [ ]:
def get_field_instru_map(instru_field_map: dict):
    '''
    Get lookup table for every variable in redcap project.
    '''
    field_instru_map = dict()
    for instru in instru_field_map:
        for field in instru_field_map[instru]:
            field_instru_map[field] = instru

    return field_instru_map

In [ ]:
def process_project_records(project_records: pd.DataFrame, field_instru_map: dict):
    '''
    1. Turn data from wide to long for easier comparison with database data
    2. Rename, sort based on patient_name
    3. Convert data types for easier comparisons
    '''
    # Wide to long
    project_records = pd.melt(project_records, id_vars=['record_id', 'redcap_repeat_instrument', 'redcap_repeat_instance'])

    # Rename for less confusion when comparing
    project_records.rename(
        columns={
            'record_id': 'patient_name',
            'redcap_repeat_instrument': 'crf_name',
            'redcap_repeat_instance': 'instance',
            'variable': 'redcap_variable',
        }, inplace=True
    )

    # Filter out 'residual rows'
    # Residual rows are defined as rows that have a repeating instrument paired with a redcap variable not belonging to it.
    # These rows do not contain the actual value of said variable.
    # TODO: figure out how to actual do this
    project_records = project_records[
        pd.notna(project_records['crf_name']) &
        project_records['redcap_variable'] == 
    ]

    # Sort
    project_records.sort_values(by='patient_name', inplace=True)

    # Data type conversion
    # object -> str
    project_records['value'] = project_records['value'].astype("string")
    project_records['redcap_variable'] = project_records['redcap_variable'].astype("string")
    project_records['crf_name'] = project_records['crf_name'].astype("string")
    project_records['patient_name'] = project_records['patient_name'].astype("string")
    
    project_records.to_excel(f'{project_name}_redcap.xlsx')

    return project_records

In [154]:
instru_field_map = get_project_instru_field_map(project)
field_instru_map = get_field_instru_map(instru_field_map)

In [64]:
project_records = get_project_records(project)
project_records

,record_id,redcap_repeat_instrument,redcap_repeat_instance,q1001,q1002___1,q1002___2,q1002___3,q1002___4,q1003,q1004,...,f_u_cr_sign,comments,f_u_status,f_u_comments,f_u_init,followup_complete,ambra_img_url,is_test,is_phantom,lab_use_only_complete
0,11001,NaN,NaN,1.0,1.0,1.0,0.0,0.0,NaN,2024-02-14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,11001,followup,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,MGS,NaN,4.0,Unchecked CTA in Scan Type because a CTA does ...,JJJ,2.0,NaN,NaN,NaN,NaN
2,11002,NaN,NaN,1.0,1.0,1.0,0.0,0.0,NaN,2024-02-24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3,11002,followup,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,MGS,signature added by VJK. Signature corrected fr...,4.0,"""Signature of Central Reader"" unanswered; JJJ....",JJJ,2.0,NaN,NaN,NaN,NaN
4,11003,NaN,NaN,1.0,1.0,1.0,0.0,0.0,NaN,2024-03-26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223,23005,NaN,NaN,1.0,1.0,1.0,1.0,0.0,NaN,2024-11-10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
224,23005,followup,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,MGS,NaN,4.0,NaN,JJJ,2.0,NaN,NaN,NaN,NaN
225,23006,NaN,NaN,1.0,1.0,1.0,1.0,0.0,NaN,2024-12-11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
226,23007,NaN,NaN,1.0,1.0,1.0,0.0,0.0,NaN,2025-01-07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [65]:
project_records = process_project_records(project_records)
project_records


,patient_name,crf_name,instance,redcap_variable,value
0,11001,<NA>,NaN,q1001,1.0
10032,11001,<NA>,NaN,q1012_w_laca_lpca___1,0.0
120157,11001,followup,1.0,l_paraclinoid_ica___1,<NA>
262201,11001,followup,1.0,l_car_bif_evt_8___1,<NA>
262200,11001,<NA>,NaN,l_car_bif_evt_8___1,0.0
...,...,...,...,...,...
278158,23007,<NA>,NaN,r_aica_evt_9___1,0.0
277931,23007,followup,1.0,r_dist_pca_evt_9___1,<NA>
277930,23007,<NA>,NaN,r_dist_pca_evt_9___1,0.0
277702,23007,<NA>,NaN,r_p3_evt_9___1,0.0


Get database data

In [8]:
db = Database(db_name)

In [9]:
def get_database_records(db: Database):
    '''Get records from database'''
    records = db.run_select_query(
        f"""
        SELECT * FROM {db_name}.CRF_Data_RedCap 
        INNER JOIN {db_name}.CRF_RedCap 
            ON {db_name}.CRF_Data_RedCap.id_crf = {db_name}.CRF_RedCap.id
        INNER JOIN {db_name}.patients
            ON {db_name}.CRF_RedCap.id_patient = {db_name}.patients.id
        """,
        column_names=True
    )
    records = pd.DataFrame.from_records(records)
    return records

In [10]:
def process_database_records(db_records: pd.DataFrame):
    '''
    1. Drop irrelevant columns and sort based on patient_name
    2. Convert data types for easier comparisons
    '''
    
    # Drop & Sort
    db_records.reset_index(inplace=True)
    db_records.drop(labels=['index','id', 'id_crf', 'record_created', 'is_phantom', 'record_updated', 'id_patient', 'patient_id', 'is_phantom'], inplace=True, axis=1)
    db_records.sort_values(by='patient_name', inplace=True)

    # Data type conversion
    # Before converting
    print(db_records.info(verbose=True))
    # object -> str
    db_records['value'] = db_records['value'].astype("string")
    db_records['redcap_variable'] = db_records['redcap_variable'].astype("string")
    db_records['crf_name'] = db_records['crf_name'].astype("string")
    db_records['patient_name'] = db_records['patient_name'].astype("string")

    # After converting
    print(db_records.info(verbose=True))
    db_records.to_excel(f'{db_name}_database.xlsx')

    return db_records

In [11]:
db_records = get_database_records(db)
db_records = process_database_records(db_records)

<class 'pandas.core.frame.DataFrame'>
Index: 186417 entries, 97013 to 3978
Data columns (total 7 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   value            186417 non-null  object 
 1   redcap_variable  186417 non-null  object 
 2   crf_name         186417 non-null  object 
 3   instance         12707 non-null   float64
 4   verified         186417 non-null  int64  
 5   deleted          186417 non-null  int64  
 6   patient_name     186417 non-null  object 
dtypes: float64(1), int64(2), object(4)
memory usage: 11.4+ MB
None
<class 'pandas.core.frame.DataFrame'>
Index: 186417 entries, 97013 to 3978
Data columns (total 7 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   value            186417 non-null  string 
 1   redcap_variable  186417 non-null  string 
 2   crf_name         186417 non-null  string 
 3   instance         12707 non-null   float64
 4   verified

In [12]:
db_records

,value,redcap_variable,crf_name,instance,verified,deleted,patient_name
97013,0,r_p3_evt_3(1),evt,NaN,1,0,11001
96245,0,l_distal_a2_evt_8(1),evt,NaN,1,0,11001
96246,0,l_distal_a2_evt_9(1),evt,NaN,1,0,11001
96247,0,l_distal_a2_evt(1),evt,NaN,1,0,11001
96248,0,l_distal_aca_evt_10(1),evt,NaN,1,0,11001
...,...,...,...,...,...,...,...
4029,0,q1010(3),ct,NaN,0,1,90003
4028,0,q1010(2),ct,NaN,0,1,90003
4027,0,q1010(1),ct,NaN,0,1,90003
4034,0,q1012_l_heub(1),ct,NaN,0,1,90003


### 2. Comparison

Map `project_records` to `db_records`.

In [50]:
def check_checkbox_redcap_var(redcap_variable: str):
    '''
    Check if `redcap_variable` is a checkbox variable.
    A checkbox variable would have two underscores before its final part.
    Example:

    checkbox_variable___1
    Two conditions:
    1. '___' is in redcap_variable
    2. '___' is the last underscore occurrence in redcap_variable
    '''
    if ('__' in redcap_variable) and (redcap_variable.rindex('_') == (redcap_variable.rindex('___') + 2)):
        return True
    return False

In [71]:
def extract_variable(redcap_variable: str):
    '''
    Extract the original variable if it is a multichoice variable.
    '''
    if check_checkbox_redcap_var(redcap_variable):
        before_choice = redcap_variable[:(redcap_variable.rindex('___'))]
        return before_choice
    return redcap_variable

In [118]:
def check_filled_instrument(record, crf_name: str, project_records: pd.DataFrame):
    '''
    Check if a particular instrument has been filled out for a record in project_records.
    '''
    instrument_complete = f'{crf_name}_complete'
    filled_out = project_records[
        (project_records['crf_name'] == crf_name) & 
        (project_records['redcap_variable'] == instrument_complete) &
        (pd.notna(project_records['value']))
    ]

    if len(filled_out) > 0:
        return True
    return False


In [ ]:
def get_unfilled_instru(record, project_records: pd.DataFrame, field_instru_map: dict):
    '''Get set of instruments that are unfilled of a specific record'''
    unfilled_instru = set()
    filled_instru = set()
    print(f'''
        Record:     {record}
        ''')

    variables = project_records['redcap_variable'].unique()

    for variable in variables:
        instru = field_instru_map[variable]

        if (instru in unfilled_instru ) or (instru in filled_instru):
            continue
        
    # for instru in instru_field_map:
    #     filled = project_records[
    #         (project_records['patient_name'] == record) &
    #         (project_records['crf_name'] == instru) 
    #         # (~project_records[pd.isna(f'{instru}_complete')])
    #     ]
    #     print(f'instru: {instru}')
    #     print(f'''filled:
    #           {filled} 
    #           ''')
    #     if len(filled) == 0:
    #         instru = instru['crf_name'].iloc[0]
    #         unfilled_instru.add(instru)

    return unfilled_instru

In [134]:
def get_record_unfilled_instru_map(project_records: pd.DataFrame, instru_field_map:dict):
    '''Get a map of {record: set of unfilled instruments}'''

    record_unfilled_instru_map = dict()
    records = project_records['patient_name'].unique()

    for record in records:
        unfilled_instru = get_unfilled_instru(record, project_records, instru_field_map)
        record_unfilled_instru_map[record] = unfilled_instru

    return record_unfilled_instru_map

In [ ]:
record_unfilled_instru_map = get_record_unfilled_instru_map(project_records, instru_field_map)


        Record:     11001
        
instru: ct
filled:
              Empty DataFrame
Columns: [patient_name, crf_name, instance, redcap_variable, value]
Index: [] 
              


TypeError: string indices must be integers, not 'str'

In [ ]:
print(f'instru field map: {instru_field_map}')
print(f'field instru map: {field_instru_map}')
print(f'record unfilled instru map: {record_unfilled_instru_map}')

{'ct': ['q1001',
  'q1002',
  'q1003',
  'q1004',
  'q1005',
  'q1006',
  'q1007',
  'q1008',
  'q7001',
  'q7002',
  'q7003',
  'q1009',
  'q1010',
  'q1011',
  'q1012_table',
  'q1012_raca',
  'q1012_r_heub',
  'q1012_rmca',
  'q1012_r_len_str',
  'q1012_r_ant_ch_art',
  'q1012_r_pca',
  'q1012_r_post_cer_art',
  'q1012_w_raca_rmca',
  'q1012_w_rmca_rpca',
  'q1012_w_raca_rpca',
  'q1012_laca',
  'q1012_l_heub',
  'q1012_l_mca',
  'q1012_l_len_str',
  'q1012_l_ant_ch_art',
  'q1012_l_pca',
  'q1012_l_post_cer_art',
  'q1012_w_laca_lmca',
  'q1012_w_lmca_lpca',
  'q1012_w_laca_lpca',
  'q1013_table',
  'q1013_r_pica',
  'q1013_r_aica',
  'q1013_r_sca',
  'q1013_r_pca_ct',
  'q1013_bas_per',
  'q1013_l_pica',
  'q1013_l_aica',
  'q1013_l_sca',
  'q1013_l_pca_ct',
  'q1015',
  'q1016',
  'hyperdense_table',
  'q1017_r_car_term',
  'q1017_r_mca',
  'q1017_bas',
  'q1017_r_aca_iv',
  'q1017_r_pca_iv',
  'q1017_other_box',
  'q1017_other',
  'q1017_l_car_term',
  'q1017_l_mca',
  'q1017_l_

In [112]:
field_instru_map

{'q1001': 'ct',
 'q1002': 'ct',
 'q1003': 'ct',
 'q1004': 'ct',
 'q1005': 'ct',
 'q1006': 'ct',
 'q1007': 'ct',
 'q1008': 'ct',
 'q7001': 'ct',
 'q7002': 'ct',
 'q7003': 'ct',
 'q1009': 'ct',
 'q1010': 'ct',
 'q1011': 'ct',
 'q1012_table': 'ct',
 'q1012_raca': 'ct',
 'q1012_r_heub': 'ct',
 'q1012_rmca': 'ct',
 'q1012_r_len_str': 'ct',
 'q1012_r_ant_ch_art': 'ct',
 'q1012_r_pca': 'ct',
 'q1012_r_post_cer_art': 'ct',
 'q1012_w_raca_rmca': 'ct',
 'q1012_w_rmca_rpca': 'ct',
 'q1012_w_raca_rpca': 'ct',
 'q1012_laca': 'ct',
 'q1012_l_heub': 'ct',
 'q1012_l_mca': 'ct',
 'q1012_l_len_str': 'ct',
 'q1012_l_ant_ch_art': 'ct',
 'q1012_l_pca': 'ct',
 'q1012_l_post_cer_art': 'ct',
 'q1012_w_laca_lmca': 'ct',
 'q1012_w_lmca_lpca': 'ct',
 'q1012_w_laca_lpca': 'ct',
 'q1013_table': 'ct',
 'q1013_r_pica': 'ct',
 'q1013_r_aica': 'ct',
 'q1013_r_sca': 'ct',
 'q1013_r_pca_ct': 'ct',
 'q1013_bas_per': 'ct',
 'q1013_l_pica': 'ct',
 'q1013_l_aica': 'ct',
 'q1013_l_sca': 'ct',
 'q1013_l_pca_ct': 'ct',
 'q1015

In [124]:
def map_project_to_db(project_records: pd.DataFrame, db_records: pd.DataFrame):
    '''
    Iterate through each project_records and check if
    that data is found in db_records
    '''

    logging.basicConfig(
        filename=f'redcap-db.log',
        filemode='w',
        level=logging.INFO
        )

    logger.info('===========================')
    for row in project_records.itertuples():
        patient_name = row.patient_name
        original_variable = row.redcap_variable
        redcap_variable = row.redcap_variable
        instance = row.instance
        crf_name = row.crf_name
        value = row.value

        # If the value is an int, it might be stored as a float in the project_records in df,
        # but an int in db_records. In this case, convert the float into the int.
        try:
            value = str(int(float(value)))
        except Exception:
            logger.info(f'Value {value} is not integer')


        # If the variable is a checkbox variable, then in project_records it would look like 
        # checkbox__1 but in db_records it is stored as checkbox(1). Convert it into 
        # the db convention for easier viewing.
        if check_checkbox_redcap_var(redcap_variable):
            master_variable = extract_variable(redcap_variable)
            before_choice = redcap_variable[:(redcap_variable.rindex('___'))]
            choice = redcap_variable[(redcap_variable.rindex('_') + 1):]
            redcap_variable = f'{before_choice}({choice})'

            # If the variable belongs to a non-filled out instrument, then skip.
            # project_records have values of variables belonging to unfilled instruments because
            # of them being having a default on REDCap. These values are not of concern.
                
            actual_crf = field_instru_map[master_variable]
            if not check_filled_instrument(patient_name, actual_crf, project_records):
                logging.info(f'Instrument {actual_crf} not filled for patient {patient_name}')
                continue
        else:
            actual_crf = field_instru_map[redcap_variable]
            if not check_filled_instrument(patient_name, actual_crf, project_records):
                logging.info(f'Instrument {actual_crf} not filled for patient {patient_name}')
                continue
        # Type of each relevant variable is printed out for
        # reference when comparison. For example, '1.0' != 1.0,
        # but for comparison purposes we treat these two as the same.
        logger.info(f'''
            ##########################
            #   Subject:     {patient_name}
            #   Instance:    {instance}
            #   OG Var:      {original_variable}
            $   Variable:    {redcap_variable}
            #   CRF_Name:    {crf_name}
            #   Value:       {value}
            ##########################
            ''')

        # If a repeating instrument
        if pd.notna(instance):
            logger.info('--------REPEATING')
            
            # If the variable is not actually in the repeating instrument, then skip
            if master_variable not in instru_field_map[crf_name]:
                logger.info(f'Master variable:    {master_variable}')
                logger.info(f'CRF:                {crf_name}')
                continue
            
            db_record = db_records[
                (db_records['patient_name'] == patient_name) &
                (db_records['redcap_variable'] == redcap_variable) &
                (db_records['instance'] == instance) &
                (db_records['crf_name'] == crf_name) &
                (db_records['value'] == value) &
                (db_records['deleted'] == 0) # Value must be not deleted in DB
            ]
            if len(db_record) == 0:
                potential_rows = db_records[
                    (db_records['patient_name'] == patient_name) &
                    (db_records['redcap_variable'] == redcap_variable) &
                    (db_records['crf_name'] == crf_name) 
                ]
                logger.info(
                    f'''
                    Not found in DB. Potential rows:

                    {potential_rows}
                    ''')
                return
            else:
                logger.info(f'Length of db_record: {len(db_record)}')
                logger.info('db_record:')
                logger.info(f'\n{db_record}')
        
        # If not a repeating instrument
        else:
            db_record = db_records[
                (db_records['patient_name'] == patient_name) &
                (db_records['redcap_variable'] == redcap_variable) &
                
                # Use .isna() instead of instance == instance because NaN != NaN is True.
                # https://stackoverflow.com/questions/10034149/why-is-nan-not-equal-to-nan
                
                (pd.isna(db_records['instance'])) & 
                (db_records['value'] == value) &
                (db_records['deleted'] == 0) # Value must be not deleted in DB
            ]
            if len(db_record) == 0:
                potential_rows = db_records[
                    (db_records['patient_name'] == patient_name) &
                    # (db_records['redcap_variable'] == redcap_variable) &
                    (pd.isna(db_records['instance']))
                ]

                logger.info(
                    f'''
                    Not found in DB. Potential rows:
                    
                    {potential_rows}
                    ''')
                return
            else:
                logger.info(f'Length of record found: {len(db_record)}')
                logger.info('db_record:')
                logger.info(f'\n{db_record}')


In [125]:
map_project_to_db(project_records, db_records)

In [48]:
tring = 'hel__hi'
# h  e  l   _    _   h   i
# 0  1  2   3    4   5   6

tring.rindex('__')

3